# Student grades

At the end of the year we can ask for an excel-sheet containing all grades for all students of ITF (and ACS, and ELO-ICT and the "WES", which is "working and studying" for people who come to get a degree while still working four days a week). This sheet contains all students, their courses and their grades.

Unfortunately, we can't pass this information on. Even more than not following the GDPR (and thus illegal) it would show an appalling lack in tact.

Therefore we changed some things
- All student ID's (r-numbers) have been changed into an equally long random number
- All names have been changed into the names of actors
- All grades have been changed into a letter using the function in the next code-block

Student ID's and names have always been changed into the same actor. So if student X became Matt Damon in 20-21, the same student will be Matt Damon in 21-22.


In [ ]:
# the function used to change the grades:

def replace_grade(grade):
    if grade in ['NA', '#']: # NA = student did not take exam, but could have and # = no exam
        return grade
    if grade == 'G': # Pass/fail-course, passed
        return "PASS"
    if grade == 'NG': # Pass/fail-course, not passed
        return "FAIL"
    grade = float(grade) # Grade is a number, so student took exam and got some grades
    if grade == 0:
        return 'ZERO'
    if grade >= 17:
        return 'A'
    if grade >= 14:
        return 'B'
    if grade >= 10:
        return 'C'
    if grade >= 8: 
        return 'D'
    return 'F'

The files are all csv-files. You can open them as follows:

In [ ]:
# todo